In [1]:
%matplotlib inline
import rasterio
import matplotlib.pyplot as plt
import gdal
import numpy as np
import timeit

In [2]:
# Function list
def band_average(raster, bands, icethickness):
    band_average = np.zeros((bands.max()+1, 1))
    for n in range(bands.max()+1):
        raster_single = raster.copy()
        raster_single[bands != n] = np.nan
        raster_single[(icethickness == 0) & (emergence_velocity == 0)] = np.nan
        band_average[n,0] = np.nanmean(raster_single)
    return band_average
    

def emergence_pixels(vel_x_raw, vel_y_raw, vel_total, vel_min, vel_depth_avg_factor, icethickness, bands):
    """ Compute the emergence velocity using an ice flux approach
    """
    # Modify vel_y by multiplying velocity by -1 such that matrix operations agree with flow direction
    #    Specifically, a negative y velocity means the pixel is flowing south.
    #    However, if you were to subtract that value from the rows, it would head north in the matrix.
    #    This is due to the fact that the number of rows start at 0 at the top.
    #    Therefore, multipylying by -1 aligns the matrix operations with the flow direction
    vel_y = vel_y_raw * -1 * vel_depth_avg_factor
    vel_x = vel_x_raw * vel_depth_avg_factor
    # Compute the initial volume
    volume_initial = icethickness * pixel_size**2
    # Quality control options:
    # Apply a border based on the max specified velocity to prevent errors associated with pixels going out of bounds
    if option_border == 1:
        border = int(max_velocity / pixel_size)
        for r in range(vel_x.shape[0]):
            for c in range(vel_x.shape[1]):
                if (r < border) | (r >= vel_x.shape[0] - border) | (c < border) | (c >= vel_x.shape[1] - border):
                    vel_x[r,c] = 0
                    vel_y[r,c] = 0
    # Minimum/maximum velocity bounds
    if option_minvelocity == 1:
        vel_x[vel_total < vel_min] = 0
        vel_y[vel_total < vel_min] = 0
    if option_maxvelocity == 1:
        vel_x[vel_total > max_velocity] = 0
        vel_y[vel_total > max_velocity] = 0
    # Remove clusters of high velocity on stagnant portions of glaciers due to feature tracking of ice cliffs and ponds
    if option_stagnantbands == 1:
        vel_x[bands <= stagnant_band] = 0
        vel_y[bands <= stagnant_band] = 0        
    # Compute displacement in units of pixels
    vel_x_pix = vel_x / pixel_size
    vel_y_pix = vel_y / pixel_size
    # Compute the displacement and fraction of pixels moved for all columns (x-axis)
    # col_x1 is the number of columns to the closest pixel receiving ice [ex. 2.6 returns 2, -2.6 returns -2]
    #    int() automatically rounds towards zero
    col_x1 = vel_x_pix.astype(int)
    # col_x2 is the number of columns to the further pixel receiving ice [ex. 2.6 returns 3, -2.6 returns -3]
    #    np.sign() returns a value of 1 or -1, so it's adding 1 pixel away from zero
    col_x2 = (vel_x_pix + np.sign(vel_x_pix)).astype(int)
    # rem_x2 is the fraction of the pixel that remains in the further pixel (col_x2) [ex. 2.6 returns 0.6, -2.6 returns 0.6]
    #    np.sign() returns a value of 1 or -1, so multiplying by that ensures you have a positive value
    #    then when you take the remainder using "% 1", you obtain the desired fraction
    rem_x2 = np.multiply(np.sign(vel_x_pix), vel_x_pix) % 1
    # rem_x1 is the fraction of the pixel that remains in the closer pixel (col_x1) [ex. 2.6 returns 0.4, -2.6 returns 0.4]
    rem_x1 = 1 - rem_x2
    # Repeat the displacement and fraction computations for all rows (y-axis)
    row_y1 = vel_y_pix.astype(int)
    row_y2 = (vel_y_pix + np.sign(vel_y_pix)).astype(int)
    rem_y2 = np.multiply(np.sign(vel_y_pix), vel_y_pix) % 1
    rem_y1 = 1 - rem_y2
    # Compute the mass flux for each pixel
    volume_final = np.zeros(volume_initial.shape)
    for r in range(vel_x.shape[0]):
        for c in range(vel_x.shape[1]):
            volume_final[r+row_y1[r,c], c+col_x1[r,c]] = (
                volume_final[r+row_y1[r,c], c+col_x1[r,c]] + rem_y1[r,c]*rem_x1[r,c]*volume_initial[r,c]
                )
            volume_final[r+row_y2[r,c], c+col_x1[r,c]] = (
                volume_final[r+row_y2[r,c], c+col_x1[r,c]] + rem_y2[r,c]*rem_x1[r,c]*volume_initial[r,c]
                )
            volume_final[r+row_y1[r,c], c+col_x2[r,c]] = (
                volume_final[r+row_y1[r,c], c+col_x2[r,c]] + rem_y1[r,c]*rem_x2[r,c]*volume_initial[r,c]
                )
            volume_final[r+row_y2[r,c], c+col_x2[r,c]] = (
                volume_final[r+row_y2[r,c], c+col_x2[r,c]] + rem_y2[r,c]*rem_x2[r,c]*volume_initial[r,c]
                )
    # Check that mass is conserved (threshold = 0.1 m x pixel_size**2)
    print('Mass is conserved?', (volume_final.sum() - volume_initial.sum()) < 0.1 * pixel_size**2)
    print(volume_final.sum() - volume_initial.sum())
    # Final ice thickness
    icethickness_final = volume_final / pixel_size**2
    # Emergence velocity
    emergence_velocity = icethickness_final - icethickness
    return emergence_velocity


def export_raster(raster, output_filename, ds, NoData_value):
    driver = gdal.GetDriverByName('GTiff')
    new_dataset = driver.Create(output_filename,
                                ds.RasterXSize,    # number of columns
                                ds.RasterYSize,    # number of rows
                                1,                  # number of bands
                                gdal.GDT_Float32)  # datatype of the raster
    new_dataset.SetProjection(ds.GetProjection())
    new_dataset.SetGeoTransform(ds.GetGeoTransform())
    # Now we need to set the band's nodata value to -1
    new_band = new_dataset.GetRasterBand(1)
    new_band.SetNoDataValue(NoData_value)
    # And finally, let's write our NDVI array.
    new_band.WriteArray(raster)
    

def import_raster(raster_fn):
    """Open raster and obtain the values in its first band as an array
    Output: array of raster values
    """
    # open raster dataset
    raster_ds = gdal.Open(raster_fn)
    # extract band information and get values
    raster_band = raster_ds.GetRasterBand(1)
    raster_values = raster_band.ReadAsArray()
    return raster_values

In [3]:
# ----- Input for Ngozumpa Glacier ----------
# Filenames associated with .tifs for the x and y velocities and their uncertainty, the ice thickness, and the bands
vel_x_fn = '../DShean_20171025/Ngoz_tiffs/ngozumpa_20112016_vx_clippedV2.tif'
vel_y_fn = '../DShean_20171025/Ngoz_tiffs/ngozumpa_20112016_vy_clippedV2.tif'
vel_x_std_fn = '../DShean_20171025/Ngoz_tiffs/Ngoz_dshean_vx_std_clipV2.tif'
vel_y_std_fn = '../DShean_20171025/Ngoz_tiffs/Ngoz_dshean_vy_std_clipV2.tif'
icethickness_fn = '../DShean_20171025/Ngoz_tiffs/Ngoz_IceThickness_Huss.tif'
bands_fn = '../DShean_20171025/Ngoz_tiffs/Ngoz_Bands_600m_ClippedV2.tif'
# Additional input
mc_simulations = 1000
pixel_size = 30
# Minimum velocity mean, which is used to determine where the glacier is stagnant based on velocities from off-glacier terrain.
min_velocity_mean = 5.218
min_velocity_std = 4.352
# Maximum velocity
max_velocity = 200
# Bias associated with ice thickness estimates
icethickness_uncertainty_percentage = -21
# Velocity depth average factor
vel_depth_avg_factor_min = 0.8
vel_depth_avg_factor_max = 0.8
# Band below which the glacier is considered to be stagnant (this avoids errors associated with artifacts from cliffs and ponds)
#  enter 0 if no portion of the glacier is stagnant
stagnant_band = 11

# ----- Quality Control Options ----------
# Filter min/max velocity
option_minvelocity = 1
option_maxvelocity = 1
#  > 1 (default) - apply limits specified in model input
#  > 0 - do not apply limits
option_border = 1
#  > 1 (default) - apply the border to prevent errors
#  > 0 - do not apply border, assumes errors near edges are avoided elsewhere
option_stagnantbands = 1
#  > 1 (default) - remove velocities from stagnant bands that are caused by cliffs and ponds
#  > 0 - do not remove the clusters of higher velocities on the tongues of the glaciers

In [4]:
# Open raster for properties (will be used to export tif)
ds = gdal.Open(vel_x_fn)
# Import rasters
vel_x = import_raster(vel_x_fn)
vel_y = import_raster(vel_y_fn)
# Imja estimates using Deheqc, which doesn't have the stdev files
try:
    vel_x_std = import_raster(vel_x_std_fn)
    vel_y_std = import_raster(vel_y_std_fn)
except:
    vel_x_std = np.zeros((vel_x.shape[0], vel_x.shape[1])) + 1.61
    vel_y_std = np.zeros((vel_x.shape[0], vel_x.shape[1])) + 1.61
icethickness = import_raster(icethickness_fn)
bands = import_raster(bands_fn)

In [2]:
# MONTE CARLO SIMULATIONS OF EMERGENCE VELOCITY
# Compute the total velocity
vel_total = (vel_x**2 + vel_y**2)**0.5
#   The total velocity correction, i.e., the minimum velocity based on the total velocity, must be applied prior to the 
#   velocity being adjusted for uncertainties because those uncertainties would change the minimum velocity threshold
#   to the exact same extent.  Hence, the total velocity is computed outside the MC simulations here.
# Generate normal random numbers (mean 0, stdev 1) to include vx and vy uncertainties
mc_vm_uncertainty = np.random.normal(0, 1, (1, mc_simulations))
# Generate normal distribution for minimum velocity threshold
mc_vmin_uncertainty = np.random.normal(min_velocity_mean, min_velocity_std, (1, mc_simulations))
# Generate uniform distribution for ice thickness uncertainty between Huss and -21% underestimation
mc_icethickness_uncertainty = np.random.uniform(icethickness_uncertainty_percentage, 0, (1, mc_simulations))
# Generate uniform distribution for ratio of surface to mean velocity
mc_vsfc_ratio_uncertainty = np.random.uniform(vel_depth_avg_factor_min, vel_depth_avg_factor_max, (1, mc_simulations))
# Run the Monte Carlo simulations
for sim in range(mc_simulations):  
    print('Simulation:', sim)
    # Compute velocities at each pixel according to uncertainty
    mc_vel_x = vel_x + (mc_vm_uncertainty[0, sim] * vel_x_std)
    mc_vel_y = vel_y + (mc_vm_uncertainty[0, sim] * vel_y_std)
    # Minimum velocity including uncertainty
    mc_vmin = mc_vmin_uncertainty[0, sim]
    # Compute ice thicknesses based on uncertainty
    mc_icethickness = icethickness / (1 + mc_icethickness_uncertainty[0, sim]/100)
    #  percent_uncertainty = (x2 - x1) / x1 * 100
    #  percent_uncertainty is the uncertainty (mean = -21%, stdev = 60%)
    #  x2 is the modeled ice thickness (Huss and Farinotti, 2012)
    #  We want to solve for x1, i.e., the ice thickness with the error included
    #    x1 = x2 / (1 + percent_uncertainty / 100)
    mc_vsfc_ratio = mc_vsfc_ratio_uncertainty[0, sim]
    # Emergence Velocity Calculations for each pixel
    emergence_velocity = emergence_pixels(mc_vel_x, mc_vel_y, vel_total, mc_vmin, mc_vsfc_ratio, mc_icethickness, bands)
    # Emergence Velocity Calculations for each band (average the pixel values)
    bands_emergence = band_average(emergence_velocity, bands, icethickness)
    # Record the results
    if sim == 0:
        mc_bands_emergence = bands_emergence
    else:
        mc_bands_emergence = np.concatenate((mc_bands_emergence, bands_emergence), axis=1)
# Calculate statistics of Monte Carlo simulatuions
mc_bands_emergence_wstats = np.zeros((mc_bands_emergence.shape[0],mc_bands_emergence.shape[1]+6))
# Column 01 = band number
bands_col = np.arange(bands.max()+1)
mc_bands_emergence_wstats[:,0] = bands_col
# Column 02 - 06 = median, mean, stdev, min, max
mc_bands_emergence_wstats[:,1] = np.median(mc_bands_emergence, axis=1)
mc_bands_emergence_wstats[:,2] = np.mean(mc_bands_emergence, axis=1)
mc_bands_emergence_wstats[:,3] = np.std(mc_bands_emergence, axis=1)
mc_bands_emergence_wstats[:,4] = np.min(mc_bands_emergence, axis=1)
mc_bands_emergence_wstats[:,5] = np.max(mc_bands_emergence, axis=1)
# Columns 7 - ... = MC simulations
mc_bands_emergence_wstats[:,6:mc_bands_emergence.shape[1]+7] = mc_bands_emergence

In [6]:
# Save the emergence velocity to a text file.  Specify the filename below.
np.savetxt('../Output/emergence_Ngoz_mcstat.txt', mc_bands_emergence_wstats)